<a href="https://colab.research.google.com/github/ImitatedSky/Python_AI/blob/main/news_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

# data download from https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news/data

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/dataset') #切換該目錄

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding
from keras.layers import SpatialDropout1D, Conv1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
cnbc_df =  pd.read_csv("cnbc_sentiment.csv")
guardian_df =  pd.read_csv("guardian_sentiment.csv")
reuters_df =  pd.read_csv("reuters_sentiment.csv")


In [5]:
cnbc_df

,Unnamed: 0,Headlines,Time,Sentiment
0,0,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020",5
1,1,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020",4
2,3,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020",5
3,4,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020",5
4,5,Wall Street delivered the 'kind of pullback I'...,"7:36 PM ET Thu, 16 July 2020",4
...,...,...,...,...
2795,3075,Markets lack Christmas cheer,"10:15 AM ET Tue, 26 Dec 2017",1
2796,3076,Cramer Remix: The biggest mistake you can make...,"11:12 AM ET Thu, 20 Sept 2018",1
2797,3077,Cramer says owning too many stocks and too lit...,"7:07 PM ET Fri, 22 Dec 2017",1
2798,3078,Cramer: I helped investors through the 2010 fl...,"7:07 PM ET Fri, 22 Dec 2017",4


In [6]:
guardian_df

,Unnamed: 0,Time,Headlines,Sentiment
0,0,18-Jul-20,Johnson is asking Santa for a Christmas recovery,4
1,1,18-Jul-20,‘I now fear the worst’: four grim tales of wor...,1
2,2,18-Jul-20,Five key areas Sunak must tackle to serve up e...,5
3,3,18-Jul-20,Covid-19 leaves firms ‘fatally ill-prepared’ f...,1
4,4,18-Jul-20,The Week in Patriarchy \n\n\n Bacardi's 'lad...,5
...,...,...,...,...
17795,17795,17-Dec-17,How investing in solar energy can create a bri...,5
17796,17796,17-Dec-17,Poundland suppliers hit by insurance downgrade,1
17797,17797,17-Dec-17,Cryptocurrencies: City watchdog to investigate...,4
17798,17798,17-Dec-17,Unilever sells household name spreads to KKR f...,1


In [7]:
reuters_df

,Unnamed: 0,Headlines,Time,Sentiment
0,0,TikTok considers London and other locations fo...,Jul 18 2020,3
1,1,Disney cuts ad spending on Facebook amid growi...,Jul 18 2020,1
2,2,Trail of missing Wirecard executive leads to B...,Jul 18 2020,1
3,3,Twitter says attackers downloaded data from up...,Jul 18 2020,1
4,4,U.S. Republicans seek liability protections as...,Jul 17 2020,1
...,...,...,...,...
32765,32765,Malaysia says never hired British data firm at...,Mar 20 2018,1
32766,32766,Prosecutors search Volkswagen headquarters in ...,Mar 20 2018,1
32767,32767,McDonald's sets greenhouse gas reduction targets,Mar 20 2018,5
32768,32768,Pratt & Whitney to deliver spare A320neo engin...,Mar 20 2018,5


In [8]:
df = pd.concat([cnbc_df, guardian_df, reuters_df], axis=0, ignore_index=True)
df


,Unnamed: 0,Headlines,Time,Sentiment
0,0,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020",5
1,1,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020",4
2,3,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020",5
3,4,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020",5
4,5,Wall Street delivered the 'kind of pullback I'...,"7:36 PM ET Thu, 16 July 2020",4
...,...,...,...,...
53365,32765,Malaysia says never hired British data firm at...,Mar 20 2018,1
53366,32766,Prosecutors search Volkswagen headquarters in ...,Mar 20 2018,1
53367,32767,McDonald's sets greenhouse gas reduction targets,Mar 20 2018,5
53368,32768,Pratt & Whitney to deliver spare A320neo engin...,Mar 20 2018,5


In [9]:
df = df.drop(columns=["Unnamed: 0","Time"])
df

,Headlines,Sentiment
0,Jim Cramer: A better way to invest in the Covi...,5
1,Cramer's lightning round: I would own Teradyne,4
2,"Cramer's week ahead: Big week for earnings, ev...",5
3,IQ Capital CEO Keith Bliss says tech and healt...,5
4,Wall Street delivered the 'kind of pullback I'...,4
...,...,...
53365,Malaysia says never hired British data firm at...,1
53366,Prosecutors search Volkswagen headquarters in ...,1
53367,McDonald's sets greenhouse gas reduction targets,5
53368,Pratt & Whitney to deliver spare A320neo engin...,5


In [10]:
random_index = np.random.permutation(df.index)
df = df.loc[random_index]
df

,Headlines,Sentiment
25403,Rolls-Royce to ditch targets and suspend divid...,1
33792,Fed's Rosengren: Will worry when consumers sho...,3
36978,"New blow to Boeing from engine delay, Airbus l...",5
24215,Fiat Chrysler resumes van production in Atessa...,4
41961,Daimler appoints Airbus CFO Harald Wilhelm to ...,5
...,...,...
2801,‘I now fear the worst’: four grim tales of wor...,1
42345,U.S. cracks down on foreigners dealing in Vene...,1
4014,Brexit: Gove confirms plans for checks on good...,3
38441,Carige's temporary administrators summon trade...,4


In [11]:
# Convert the dataframe to a numpy array
dataset = df.values
# 將所有的95%放下去測試
training_data_len = int(np.ceil( len(dataset) * .95 ))

print(f"總共資料筆數{len(dataset)} , 訓練資料筆數{training_data_len}")

總共資料筆數53370 , 訓練資料筆數50702


In [12]:
train_data = dataset[0:int(training_data_len), :]

x_train = train_data[:,0]
y_train = train_data[:,1]

test_data = dataset[training_data_len : , :]

x_test = test_data[:,0]
y_test = test_data[:,1]


In [13]:
pad_type = trunc_type = 'pre'
max_review_length = 36

In [14]:
# 文本向量化
from keras.preprocessing.text import Tokenizer

# 初始化一個Tokenizer
tokenizer = Tokenizer()

# 適應訓練及數列
tokenizer.fit_on_texts(x_train)
vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

# 將訓練集 測試集 轉為整數序列
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

30026


In [15]:
import tensorflow as tf
from keras.utils import to_categorical

# 再使用pad_sequences函数填充序列
x_train = pad_sequences(x_train, maxlen = max_review_length , padding=pad_type, truncating=trunc_type, value=0, dtype='int32')
x_test = pad_sequences(x_test, maxlen = max_review_length , padding=pad_type, truncating=trunc_type, value=0, dtype='int32')

x_train = tf.convert_to_tensor(x_train, dtype=tf.int32)
y_train = to_categorical(y_train)

x_test = tf.convert_to_tensor(x_test, dtype=tf.int32)
y_test = to_categorical(y_test)

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D , Dropout, Embedding

model = Sequential()

# 添加嵌入層，將整数序列轉為嵌入向量
model.add(Embedding( input_dim = vocab_size , output_dim = 100 , input_length = max_review_length ))

model.add(SpatialDropout1D( 0.2 ))

model.add(Conv1D(196, 3, activation='relu'))
model.add(Conv1D(196, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(196, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))

model.summary()

# 编译模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=8, batch_size=30)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 36, 100)           3002600   
                                                                 
 spatial_dropout1d (Spatial  (None, 36, 100)           0         
 Dropout1D)                                                      
                                                                 
 conv1d (Conv1D)             (None, 34, 196)           58996     
                                                                 
 conv1d_1 (Conv1D)           (None, 32, 196)           115444    
                                                                 
 global_max_pooling1d (Glob  (None, 196)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 196)               3

In [17]:
predictions = model.predict(x_test)
loss, acc = model.evaluate(x_test, y_test)
print(f"accuracy：{acc}")
print(f"loss：{loss}")

84/84 [==============================] - 1s 7ms/step - loss: 1.9117 - accuracy: 0.6619
accuracy：0.6619190573692322
loss：1.9116908311843872


In [18]:
_input = input("sentiment analysis contents：")
_input = tokenizer.texts_to_sequences([_input])
_input = pad_sequences(_input, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0, dtype='int32')
_input = tf.convert_to_tensor(_input, dtype=tf.int32)

#
predictions = model.predict(_input)

#
sentiment_label = np.argmax(predictions)  # 返回最大的

print("-"*80)
print(predictions)
print("prediction ：", sentiment_label)



sentiment analysis contents：Stock Market Plummets, Investors Panic
1/1 [==============================] - 0s 21ms/step
--------------------------------------------------------------------------------
[[5.1535168e-11 8.5184675e-01 6.3821452e-04 1.2924320e-03 3.6270372e-03
  1.4259556e-01]]
prediction ： 1


In [19]:
import time


filename = '../model/' +'news_sentiment_analysis'+ time.strftime("%m%d", time.localtime()) + '.h5'
print(filename)
model.save(filename)

../model/news_sentiment_analysis1108.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
